<a href="https://colab.research.google.com/github/Derinhelm/llm_tokenization/blob/main/token_aligning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Derinhelm/llm_tokenization

Cloning into 'llm_tokenization'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 29 (delta 0), reused 2 (delta 0), pack-reused 24 (from 1)
Receiving objects: 100% (29/29), 52.46 MiB | 8.04 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
import sys
sys.path.append('/content/llm_tokenization')

In [3]:
from data_classes import ConllEntry, Sentence

In [4]:
import pickle

with open('/content/llm_tokenization/treebank_test_sets/treebank_data.pickle', 'rb') as f:
    treebanks = pickle.load(f)

In [5]:
treebank_names = list(treebanks.keys())
treebank_names

['gsd', 'pud', 'syntagrus', 'poetry', 'taiga']

In [6]:
for tr in treebank_names:
  for s in treebanks[tr]:
    s.tokens = [ t for t in s.tokens if '.' not in t.id ]
    s.text = s.text.replace('\xa0', " ") # Замена неразрывного пробела

In [7]:
import pickle

with open('/content/llm_tokenization/results/sent_subtokens.pickle', 'rb') as f:
    original_sent_subtokens = pickle.load(f)

In [8]:
for m_name in original_sent_subtokens.keys():
  for tr in treebank_names:
    assert len(treebanks[tr]) == len(original_sent_subtokens[m_name][tr])

In [9]:
model_names = list(original_sent_subtokens.keys())

In [10]:
model_names

['Qwen3_4B', 'RuadaptQwen3_4B_Instruct']

In [11]:
sent_subtokens = { m_name: {} for m_name in model_names }
for m_name in model_names:
  sent_subtokens[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_id in original_sent_subtokens[m_name][tr].keys():
      d = original_sent_subtokens[m_name][tr][sent_id]
      sent_subtokens[m_name][tr][sent_id] = {}
      sent_subtokens[m_name][tr][sent_id]['token_texts'] = d[1][3:-2]
      sent_subtokens[m_name][tr][sent_id]['offsets'] = d[0]['offset_mapping'][0][3:-2] - d[0]['offset_mapping'][0][3][0]
      sent_subtokens[m_name][tr][sent_id]['subtoken_len'] = d[2][3:-2]

In [13]:
for m_name in model_names:
  for tr in treebank_names:
    for s in treebanks[tr]:
      #print(s.text.replace(" ", ""))
      #print("".join(map(lambda x: x[0].replace(" ", ""),
      #                                                             sent_subtokens[m_name][tr][s.sent_id]['subtoken_len'])))
      assert s.text.replace(" ", "") == "".join(map(lambda x: x[0].replace(" ", ""),
                                                                   sent_subtokens[m_name][tr][s.sent_id]['subtoken_len']))
      assert all([ ord(symb) != chr(65533)
                  for st in sent_subtokens[m_name][tr][s.sent_id]['subtoken_len']
                  for symb in st[0] ])

In [14]:
m_name = 'Qwen3_4B'
tr = 'gsd'
sent_id = 'test-s10'
print(sent_subtokens[m_name][tr][sent_id]['token_texts'][30:34])
print(sent_subtokens[m_name][tr]['test-s10']['subtoken_len'][30:33])

['з', 'а�', '�от', 'илось']
[('з', 1), ('абот', 2), ('илось', 1)]


# Создание статистики по количеству субтокенов в токенах дерева зависимостей


In [46]:
from collections import defaultdict

def align_tokens(gold_tokens, subtoken_lens):
    tokens1 = [ t.form.replace(" ", "") for t in gold_tokens ]
    tokens2 = [ t[0].replace(" ", "") for t in subtoken_lens ]
    token_lens2 = [ t[1] for t in subtoken_lens ]
    assert len(tokens2) == len(token_lens2)
    #print(tokens1)

    x1 = [ (i, symb) for i, t in enumerate(tokens1) for symb in t ]
    x2 = [ (i, symb) for i, t in enumerate(tokens2) for symb in t ]
    #print([e[1] for e in x1])
    #print([e[1] for e in x2])
    assert len(x1) == len(x2)

    x = []
    for i in range(len(x1)):
        assert x1[i][1] == x2[i][1]
        x.append((x1[i][0], x2[i][0]))

    fst_dict = defaultdict(set)
    snd_dict = defaultdict(set)
    for fst, snd in x:
      fst_dict[fst].add(snd)
      snd_dict[snd].add(fst)
    errors_snd = [ snd for snd in snd_dict if len(snd_dict[snd]) > 1]
    errors_fst = { fst for snd in errors_snd for fst in snd_dict[snd] }

    res = []
    for i, t in enumerate(treebanks[tr][sent_i].tokens):
      if i not in errors_fst:
          res.append((tokens1[i].strip(), [ (tokens2[j].strip(), token_lens2[j]) for j in fst_dict[i]]))
      else:
          res.append((tokens1[i], None, 0))
    return res

In [47]:
# Проверка на обычном предложении
m_name = 'Qwen3_4B'
tr = 'syntagrus'
sent_i = 8475
sent_id = '2020_RFFIMoskva_2042.xml_418'
align_tokens(treebanks[tr][sent_i].tokens, sent_subtokens[m_name][tr][sent_id]['subtoken_len'])

[('Я', [('Я', 1)]),
 ('посмотрел', [('по', 1), ('смотр', 1), ('ел', 1)]),
 ('на', [('на', 1)]),
 ('часы', [('час', 1), ('ы', 1)]),
 ('.', [('.', 1)])]

In [48]:
# Проверка на составном субтокене (абот - 2 субтокена)
m_name = 'Qwen3_4B'
tr = 'gsd'
sent_id = 'test-s10'
sent_i = 9
align_tokens(treebanks[tr][sent_i].tokens, sent_subtokens[m_name][tr][sent_id]['subtoken_len'])[10:14]

[(',', [(',', 1)]),
 ('правительство', [('правитель', 1), ('ство', 1)]),
 ('озаботилось', [('илось', 1), ('о', 1), ('з', 1), ('абот', 2)]),
 ('её', [('её', 1)])]

In [50]:
# Проверка на предложении, в котором субтокен '",' принадлежит двум эталонным токенам
m_name = 'Qwen3_4B'
tr = 'taiga'
sent_i = 864
sent_id = '7249'
align_tokens(treebanks[tr][sent_i].tokens, sent_subtokens[m_name][tr][sent_id]['subtoken_len'])

[('ресурс', [('ресурс', 1)]),
 ('не', [('не', 1)]),
 ('"', [('"', 1)]),
 ('воз', [('воз', 1)]),
 ('и', [('и', 1)]),
 ('ныне', [('н', 1), ('ы', 1), ('не', 1)]),
 ('там', [('там', 1)]),
 ('"', None, 0),
 (',', None, 0),
 ('а', [('а', 1)]),
 ('превратился', [('пр', 1), ('ев', 1), ('рат', 1), ('ился', 1)]),
 ('в', [('в', 1)]),
 ('мощный', [('мощ', 1), ('ный', 1)]),
 ('бульдозер', [('б', 1), ('уль', 1), ('д', 1), ('оз', 1), ('ер', 1)]),
 ('.', [('.', 1)])]

In [51]:
gold_token_stat = { m_name: {} for m_name in model_names }
for m_name in model_names:
  gold_token_stat[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_i, s in enumerate(treebanks[tr]):
      gold_token_stat[m_name][tr][s.sent_id] = align_tokens(
          treebanks[tr][sent_i].tokens, sent_subtokens[m_name][tr][s.sent_id]['subtoken_len'])

# Статистика по количеству субтокенов в эталонном токене

In [71]:
subtoken_amount = { m_name: {} for m_name in model_names }
for m_name in model_names:
  subtoken_amount[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_i, s in enumerate(treebanks[tr]):
      subtoken_amount[m_name][tr][s.sent_id] = [ (d[0], sum(map(lambda x: x[1], d[1])) if d[1] != None else None)
                                                        for d in gold_token_stat[m_name][tr][s.sent_id]]

In [75]:
subtoken_amount['Qwen3_4B']['taiga']['7249']

[('ресурс', 1),
 ('не', 1),
 ('"', 1),
 ('воз', 1),
 ('и', 1),
 ('ныне', 3),
 ('там', 1),
 ('"', None),
 (',', None),
 ('а', 1),
 ('превратился', 4),
 ('в', 1),
 ('мощный', 2),
 ('бульдозер', 5),
 ('.', 1)]

In [76]:
subtoken_amount['Qwen3_4B']['gsd']['test-s10']

[('Узнав', 3),
 ('о', 1),
 ('её', 1),
 ('существовании', 3),
 ('на', 1),
 ('допросе', 3),
 ('благовещенского', 5),
 ('диакона', 3),
 ('Федора', 3),
 ('Иванова', 3),
 (',', 1),
 ('правительство', 2),
 ('озаботилось', 5),
 ('её', 1),
 ('конфискацией', 6),
 (',', 1),
 ('со', 1),
 ('всеми', 2),
 ('подготовленными', 3),
 ('списками', 2),
 (',', 1),
 ('и', 1),
 ('поручило', 3),
 ('написать', 3),
 ('на', 1),
 ('неё', 2),
 ('опровержение', 4),
 ('газскому', 4),
 ('митр', 3),
 ('.', 1)]

In [77]:
model_names

['Qwen3_4B', 'RuadaptQwen3_4B_Instruct']

In [78]:
m_name1 = 'Qwen3_4B'
m_name2 = 'RuadaptQwen3_4B_Instruct'

In [82]:
pair_stat = { tr: [] for tr in treebank_names }
for tr in treebank_names:
  for s in treebanks[tr]:
    assert len(subtoken_amount[m_name1][tr][s.sent_id]) == len(subtoken_amount[m_name2][tr][s.sent_id])

    for i in range(len(subtoken_amount[m_name1][tr][s.sent_id])):
        assert subtoken_amount[m_name1][tr][s.sent_id][i][0] == subtoken_amount[m_name2][tr][s.sent_id][i][0]

        pair_stat[tr].append((s.sent_id, i
                        , subtoken_amount[m_name1][tr][s.sent_id][i][1], subtoken_amount[m_name2][tr][s.sent_id][i][1]))

In [94]:
# m_name1, m_name2
d = {}
for (subtoken_len1, subtoken_len2), pair_amount in Counter([(x[2], x[3]) for x in pair_stat[tr] if x[2] != None and x[3] != None ]).items():
  if subtoken_len1 in d:
    d[subtoken_len1][subtoken_len2] = pair_amount
  else:
    d[subtoken_len1] = { subtoken_len2: pair_amount }

In [97]:
d

{3: {3: 436, 1: 585, 2: 567, 4: 8},
 2: {1: 1143, 3: 20, 2: 505},
 1: {1: 5313, 2: 8, 3: 9},
 5: {2: 60, 4: 93, 5: 33, 1: 62, 3: 104, 6: 2},
 4: {1: 263, 3: 316, 2: 298, 4: 140, 5: 2, 6: 1},
 11: {11: 1, 7: 1},
 10: {10: 4, 6: 1},
 6: {3: 22, 2: 15, 4: 23, 5: 19, 6: 4, 1: 13, 9: 1, 7: 1},
 7: {6: 4, 5: 9, 7: 7, 3: 1, 8: 1, 4: 5, 2: 2},
 8: {7: 2, 6: 5, 8: 3, 2: 1, 4: 2, 5: 2, 1: 1},
 9: {7: 1, 5: 2},
 14: {14: 1}}

In [96]:
import pandas as pd
pd.DataFrame(d)

,3,2,1,5,4,11,10,6,7,8,9,14
3,436.0,20.0,9.0,104.0,316.0,NaN,NaN,22.0,1.0,NaN,NaN,NaN
1,585.0,1143.0,5313.0,62.0,263.0,NaN,NaN,13.0,NaN,1.0,NaN,NaN
2,567.0,505.0,8.0,60.0,298.0,NaN,NaN,15.0,2.0,1.0,NaN,NaN
4,8.0,NaN,NaN,93.0,140.0,NaN,NaN,23.0,5.0,2.0,NaN,NaN
5,NaN,NaN,NaN,33.0,2.0,NaN,NaN,19.0,9.0,2.0,2.0,NaN
6,NaN,NaN,NaN,2.0,1.0,NaN,1.0,4.0,4.0,5.0,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,7.0,2.0,1.0,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [85]:
from collections import Counter
for tr in treebank_names:
  x = Counter([(str(x[2]) if x[2] < 4 else "3+", str(x[3] if x[3] < 4 else "3+" ))
                for x in pair_stat[tr] if x[2] != None and x[3] != None])

  print(tr, "\n", [(k, v) for (k, v) in x.items() if k[0] != k[1]])

gsd 
 [(('3', '2'), 574), (('3+', '2'), 536), (('2', '1'), 1254), (('3+', '3'), 546), (('3', '1'), 938), (('1', '2'), 20), (('3+', '1'), 627), (('3', '3+'), 14), (('2', '3'), 9), (('2', '3+'), 1), (('1', '3'), 4)]
pud 
 [(('2', '1'), 2448), (('3', '1'), 1759), (('3', '2'), 797), (('3+', '1'), 1109), (('3+', '2'), 897), (('3+', '3'), 735), (('2', '3'), 4), (('1', '2'), 2), (('3', '3+'), 4)]
syntagrus 
 [(('2', '1'), 19844), (('3+', '1'), 7301), (('3', '1'), 13078), (('3+', '3'), 5970), (('3+', '2'), 6915), (('3', '2'), 7147), (('2', '3'), 109), (('1', '2'), 17), (('3', '3+'), 32), (('1', '3'), 3)]
poetry 
 [(('3', '2'), 802), (('3+', '3'), 494), (('3', '1'), 802), (('2', '1'), 1123), (('3+', '1'), 201), (('3+', '2'), 389), (('1', '2'), 11), (('2', '3'), 12), (('3', '3+'), 8)]
taiga 
 [(('2', '1'), 1143), (('3+', '2'), 376), (('3', '1'), 585), (('2', '3'), 20), (('3+', '1'), 339), (('3+', '3'), 443), (('1', '2'), 8), (('3', '2'), 567), (('3', '3+'), 8), (('1', '3'), 9)]


In [91]:
from collections import Counter
for tr in treebank_names:
  x = Counter([(x[2], x[3])
                for x in pair_stat[tr] if x[2] == None and x[3] != None])

  print(tr, "\n", x)

gsd 
 Counter()
pud 
 Counter()
syntagrus 
 Counter()
poetry 
 Counter()
taiga 
 Counter()


In [90]:
from collections import Counter
for tr in treebank_names:
  x = Counter([(x[2], x[3])
                for x in pair_stat[tr] if x[2] != None and x[3] == None])

  print(tr, "\n", x)

gsd 
 Counter()
pud 
 Counter()
syntagrus 
 Counter({(1, None): 8, (2, None): 8})
poetry 
 Counter()
taiga 
 Counter()


In [92]:
from collections import Counter
for tr in treebank_names:
  x = Counter([(x[2], x[3])
                for x in pair_stat[tr] if x[2] == None and x[3] == None])

  print(tr, "\n", x)

gsd 
 Counter({(None, None): 322})
pud 
 Counter({(None, None): 268})
syntagrus 
 Counter({(None, None): 2372})
poetry 
 Counter({(None, None): 30})
taiga 
 Counter({(None, None): 152})
